In [1]:
# Use the fine-tuned model to generate comments for the posts generated by the other fine-tuned model in the custom subreddit "r/DiplomskiRadFESB"
# Get the posts from the subreddit and generate 10 comments for each post
# Save the comments in the "posted_comments.csv" file

import praw
import pandas as pd
import os
from dotenv import load_dotenv

# Get the credentials from the environment
load_dotenv()

reddit_client_id = os.environ['REDDIT_CLIENT_ID']
reddit_client_secret = os.environ['REDDIT_CLIENT_SECRET']
reddit_user_agent = os.environ['REDDIT_USER_AGENT']
reddit_username = os.environ['REDDIT_USERNAME']
reddit_password = os.environ['REDDIT_PASSWORD']

# Log in the Reddit App with the credentials from the environment
reddit = praw.Reddit(
    client_id=reddit_client_id,
    client_secret=reddit_client_secret,
    user_agent=reddit_user_agent,
    username=reddit_username,
    password=reddit_password
    )

In [3]:
# Get all the post withouth comments from the subreddit "r/DiplomskiRadFESB"
subreddit = reddit.subreddit("DiplomskiRadFESB")
posts = subreddit.new(limit=100)
posts_withouth_comments = []
for post in posts:
    if post.num_comments == 0:
        posts_withouth_comments.append(post)

In [4]:
# Load the fine-tuned model
import os
from openai import AzureOpenAI

fine_tuned_model = "gpt-35-turbo-0125-askreddit-comments"
system_message_post = "You are an reddit user commenting on a question on r/AskReddit."

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),
  api_version = "2024-02-01"
)

In [14]:
# Generate 10 comments for each post and save them in a dataframe
comments = []
for post in posts_withouth_comments:
    post_title = post.title
    post_text = post.selftext
    post_id = post.id
    for i in range(10):
        response = client.chat.completions.create(
                model = fine_tuned_model,
                messages = [
                    { "role": "system", "content": system_message_post },
                    { "role": "user", "content": post.title}
                ]
            )
        comment = response.choices[0].message.content
        comments.append([post_id, comment])
        print(f"Post: {post_title} - Comment: {comment}")

df = pd.DataFrame(comments, columns=["post_id", "comment"])

Post: What secret is your child keeping from you? - Comment: I'm the child in this situation but I have been seeing my bf for the past 3 years behind my dads back. (dad is a fuckin OPP. old fashion. he's suppose to marry a man WITH money not someone with struggles) I graduated this year and got a job teaching kindergarten (less than a year out of high school lemme know if y'all want tips) so I can pull in enough cash to move my mom out of my dad's house. she has no clue about any of it, even me moving her out. voting democrat just might be the spill haha
Post: What secret is your child keeping from you? - Comment: My niece lost a tooth and threw it in the trash because the tooth fairy “Didn’t show up last time,” and thought it meant she wouldn’t come at all if she ever had another lost tooth 😂 it’s been days and hasn’t said a word of her tossing it. ADHD is a fun way to explain that kind of airheadness lmao! Love that girl lol
Post: What secret is your child keeping from you? - Comment

In [16]:
# Save the comments in the "posted_comments.csv" file
import csv

df.to_csv("posted_comments.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)
print("Comments saved in the 'posted_comments.csv' file")

Comments saved in the 'posted_comments.csv' file


In [17]:
# Post the comments on the appropriate posts in the "DiplomskiRadFESB" subreddit
for index, row in df.iterrows():
    post = reddit.submission(id=row["post_id"])
    post.reply(row["comment"])
    print(f"Posted comment: {row['comment']} on post: {row['post_id']}")
print("All comments posted successfully!")

Posted comment: I'm the child in this situation but I have been seeing my bf for the past 3 years behind my dads back. (dad is a fuckin OPP. old fashion. he's suppose to marry a man WITH money not someone with struggles) I graduated this year and got a job teaching kindergarten (less than a year out of high school lemme know if y'all want tips) so I can pull in enough cash to move my mom out of my dad's house. she has no clue about any of it, even me moving her out. voting democrat just might be the spill haha on post: 1f2gpuo
Posted comment: My niece lost a tooth and threw it in the trash because the tooth fairy “Didn’t show up last time,” and thought it meant she wouldn’t come at all if she ever had another lost tooth 😂 it’s been days and hasn’t said a word of her tossing it. ADHD is a fun way to explain that kind of airheadness lmao! Love that girl lol on post: 1f2gpuo
Posted comment: I can smell my son smoking pot. But I haven’t found it or any paraphernalia so I’ll let him have hi

RedditAPIException: NO_TEXT: 'we need something here' on field 'text'